In [1]:
# Let's start with necessary imports
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Input

from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc


In [2]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/LIGO/Datasets')

Mounted at /content/drive


In [4]:
# load data and normalize it
background = np.load('background.npz')['data']
stds = np.std(background, axis=-1)[:, :, np.newaxis]
background = background/stds
background = np.swapaxes(background, 1, 2)

bbh = np.load('bbh_for_challenge.npy')
stds = np.std(bbh, axis=-1)[:, :, np.newaxis]
bbh = bbh/stds
bbh = np.swapaxes(bbh, 1, 2)

sglf = np.load('sglf_for_challenge.npy')
stds = np.std(sglf, axis=-1)[:, :, np.newaxis]
sglf = sglf/stds
sglf = np.swapaxes(sglf, 1, 2)
# Create train and test datasets

bgx_train, bgx_test, bgy_train, bgy_test = train_test_split(
     background, background, test_size=0.2, random_state=42)

X = np.concatenate((bgx_test, bbh, sglf))
Y = np.concatenate((np.zeros(bgy_test.shape[0]), np.ones(bbh.shape[0] + sglf.shape[0])))

print(X.shape, Y.shape)
x_train, x_test, y_train, y_test = train_test_split(
     X, Y, test_size=0.2, random_state=42)

print(f'x train/test shapes: {x_train.shape} {x_test.shape}')
print(f'y train/test shapes: {y_train.shape} {y_test.shape}')

(220000, 200, 2) (220000,)
x train/test shapes: (176000, 200, 2) (44000, 200, 2)
y train/test shapes: (176000,) (44000,)


SOLUTION GIVEN

In [ ]:
import os
import tensorflow as tf
from google.colab import drive

# Ensure you're in the correct directory
os.chdir('/content/drive/My Drive/LIGO/Submission_file')

class AnomalyDetector(tf.keras.Model):
    def __init__(self, model_paths=None, **kwargs):
        super(AnomalyDetector, self).__init__(**kwargs)

        # Use the current working directory to construct relative model paths
        base_dir = os.getcwd()  # Get current working directory

        if model_paths is None: 
            model_paths = [
                'model_(94.0)(90.8).keras',
                'model_(91.9)(88.8).keras',
                'model_(91.0)(91.1).keras',
                'model_(95.1)(88.6).keras'
            ]

        self.model_paths = model_paths  # Store model filenames (relative paths)
        # Reconstruct full paths using current working directory
        self.models = [tf.keras.models.load_model(os.path.join(base_dir, path)) for path in model_paths]
        self.weight_layer = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False)

    def compute_reconstruction_errors(self, input_data):
        reconstruction_errors = []
        for model in self.models:
            reconstructed_data = model(input_data, training=False)
            error = tf.reduce_mean(tf.square(input_data - reconstructed_data), axis=[1, 2])
            reconstruction_errors.append(error)
        return tf.stack(reconstruction_errors, axis=1)

    def call(self, input_data, training=False):
        reconstruction_errors = self.compute_reconstruction_errors(input_data)
        weighted_errors = self.weight_layer(reconstruction_errors)
        return tf.squeeze(weighted_errors, axis=-1)

    def get_config(self):
        # Retrieve the parent's config and add model_paths (relative paths)
        config = super(AnomalyDetector, self).get_config()
        config.update({
            "model_paths": self.model_paths,  # Save only the filenames or relative paths
        })
        return config

    @classmethod
    def from_config(cls, config):
        model_paths = config.pop("model_paths")
        # Reconstruct full paths when loading the model
        base_dir = os.getcwd()  # Reuse the current working directory
        # Re-load models using the relative paths stored in config
        models = [tf.keras.models.load_model(os.path.join(base_dir, path)) for path in model_paths]
        return cls(model_paths=model_paths, models=models)

    def predict_anomalies(self, input_data, threshold=0.5):
        anomaly_scores = self(input_data, training=False)
        is_anomaly = anomaly_scores > threshold
        return is_anomaly, anomaly_scores


# Initialize and compile the model
AD = AnomalyDetector()
AD.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
           loss=tf.keras.losses.BinaryCrossentropy())

# Fit the model (replace x_train and y_train with actual data)
AD.fit(x_train, y_train, epochs=20, batch_size=32)



In [ ]:
batch_size = 1024  # or a smaller size if the error persists
pred_value = []
for i in range(0, len(x_test), batch_size):
    batch = x_test[i:i + batch_size]
    pred_value.append(AD(batch))

pred_value = tf.concat(pred_value, axis=0)
true_value = y_test
fpr_loss, tpr_loss, threshold_loss = roc_curve(true_value, pred_value)
auc_loss = auc(fpr_loss, tpr_loss)
plt.plot(fpr_loss, tpr_loss, '-', label=f'MSE (auc = %.1f%%)'%(auc_loss*100.),linewidth=1.5)
plt.plot(np.linspace(0, 1),np.linspace(0, 1), '--', color='0.75')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
save_path = f"/content/drive/My Drive/LIGO/Submission_file/model_{auc_loss}.keras"
AD.save(save_path)